### 1. Libraries

In [1]:
import pandas as pd
import time
from datetime import datetime
import re

### 2. Split 54 sensors based on 'moteid' and convert into .csv files

In [ ]:
# Define column names
column_names = ['date', 'time', 'epoch', 'moteid', 'temperature', 'humidity', 'light', 'voltage']

# Read the txt file
data = []
with open('../data/raw/data.txt', 'r') as file:
    invalid_row = []
    for line in file:
        parts = line.strip().split(' ')
        if len(parts) < 8:
            invalid_row.append(parts)
        data.append(parts)

    # for row in invalid_row:
    #     print(row)

df = pd.DataFrame(data, columns=column_names)

# Group by 'moteid' and write each group to a separate CSV file
for moteid, group in df.groupby('moteid'):
    # Define a file name based on moteid
    file_name = f'../data/processed/unsorted/moteid_{moteid}.csv'
    # Write the group to a CSV file
    group.to_csv(file_name, index=False)

print("Files saved successfully!")

### 3. Combine date and time columns and sort the data based on the resulting datetime values

In [ ]:
# Define a function to clean and ensure consistent time formatting
def clean_time(time_str):
    try:
        # If the time contains microseconds, it's fine, return it as-is
        if '.' in time_str:
            return time_str
        else:
            # If the time is missing microseconds, add '.000000' for consistency
            return time_str + '.000000'
    except:
        return None

# Loop through moteid values from 1 to 54
for moteid in range(1, 55):  # Assuming moteid goes from 1 to 54
    # Construct the file path for each moteid
    file_path = f'../data/processed/unsorted/moteid_{moteid}.csv'
    
    # Read the CSV file
    try:
        df = pd.read_csv(file_path)
        
        # Apply the cleaning function to the 'time' column
        df['time'] = df['time'].apply(clean_time)
        
        # Combine 'date' and cleaned 'time' columns into a single 'datetime' column
        df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')
        
        # Drop any rows where 'datetime' could not be parsed
        df = df.dropna(subset=['datetime'])
        
        # Sort the DataFrame by 'datetime'
        df_sorted = df.sort_values(by='datetime').reset_index(drop=True)

        # # Drop the 'datetime' column after sorting
        # df_sorted = df_sorted.drop(columns=['datetime'])
        
        # Save the sorted DataFrame back to a CSV file
        output_path = f'../data/processed/sorted/moteid_{moteid}.csv'
        df_sorted.to_csv(output_path, index=False)
        
        print(f"Processed and saved: {output_path}")
    
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

In [18]:
# Define column names
column_names = ['date', 'time', 'epoch', 'moteid', 'temperature', 'humidity', 'light', 'voltage']

# Read the txt file
data = []
with open('../data/raw/data.txt', 'r') as file:
    invalid_row = []
    for line in file:
        parts = line.strip().split(' ')
        if len(parts) < 8:
            invalid_row.append(parts)
        else:
            data.append(parts)

# Create DataFrame with the valid data
df = pd.DataFrame(data, columns=column_names)

# Convert the 'epoch' column to a numeric type for sorting
df['epoch'] = pd.to_numeric(df['epoch'], errors='coerce')

# Combine 'date' and cleaned 'time' columns into a single 'datetime' column
df['datetime'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='coerce')

# Sort DataFrame based on the 'epoch' column
df_sorted = df.sort_values(by='datetime').reset_index(drop=True)

# Save the DataFrame to a CSV file
df_sorted.to_csv('../data/processed/data.csv', index=False)

print("Data has been successfully converted and saved to data.csv")

Data has been successfully converted and saved to data.csv


In [12]:
df_sorted.head(20)

,date,time,epoch,moteid,temperature,humidity,light,voltage
0,2004-03-21,19:06:25.229486,0,12,19.5964,50.5463,39.56,2.33827
1,2004-03-21,19:06:38.75603,0,11,21.772,49.2585,478.4,2.32
2,2004-03-21,19:06:50.409202,0,52,22.0856,44.879,0.92,2.34751
3,2004-03-21,19:06:25.315774,0,21,88.2062,46.2297,30.36,2.32
4,2004-03-21,19:06:33.897783,0,19,122.153,-3.91901,79.12,2.23278
5,2004-03-21,19:06:29.175673,0,47,21.8798,44.9121,0.92,2.39484
6,2004-03-21,19:07:05.908808,1,13,122.153,41.8476,57.04,2.25825
7,2004-03-21,19:07:14.467243,1,14,21.2722,48.1568,93.84,2.38522
8,2004-04-05,10:34:32.330597,2,56,25.2314,32.2293,0.0402204,2.82819
9,2004-03-30,23:24:15.419763,2,27,122.153,-3.91901,97.52,2.08393
